<a href="https://colab.research.google.com/github/Ihoold/pixelgen/blob/master/funit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from unittest import mock
import os
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.python.estimator import estimator
from google.colab import drive, auth

In [0]:
R, C = 4, 3
EXAMPLES = R * C
CHANNELS = 4

ALPHA = 0.2
BATCH_SIZE = 1024
EPOCHS = 20000
EVAL_EPOCHS = 1000
G_LR = 0.0002
D_LR = 0.0001
LR_REC = 0.1
LR_FEAT = 1
KERNEL_SIZE = 4
NUM_CLASSES = 21

MODEL_NAME = 'FUNIT'
RUN_NAME = 'FUNIT_only_disc_2'

data_train_file = 'gs://tputestingmnist/datasets/dataset_train_funit.tfrecords'
data_test_file = 'gs://tputestingmnist/datasets/dataset_test_funit.tfrecords'
MODEL_DIR = 'gs://tputestingmnist/{}/{}/'.format(MODEL_NAME, RUN_NAME)
GOOGLE_DRIVE_DIR = '/content/gdrive/My Drive/Programowanie/PixelGen/{}'.format(RUN_NAME)
TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

In [0]:
#################################### SETUP #####################################

def setup():
    drive.mount('/content/gdrive')
    auth.authenticate_user()


def upload_credentials():
    # Upload credentials to TPU.
    with tf.Session(TF_MASTER) as sess:    
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

In [0]:
################################# DATA INPUT ###################################

def parser(serialized_example):
        """Parses a single example into image and label tensors."""
        features = tf.parse_single_example(
            serialized_example,
            features={
                'image_content': tf.FixedLenFeature([], tf.string),
                'image_style': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64) 
            })

        content_image = tf.decode_raw(features['image_content'], tf.uint8)
        content_image.set_shape([48 * 48 * 4])
        content_image = tf.reshape(content_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        content_image = tf.cast(content_image, dtype=tf.float32) / 127.5 - 1

        style_image = tf.decode_raw(features['image_style'], tf.uint8)
        style_image.set_shape([48 * 48 * 4])
        style_image = tf.reshape(style_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        style_image = tf.cast(style_image, dtype=tf.float32) / 127.5 - 1
        
        label = features['label']

        return {'style_images': style_image, 'content_images': content_image, 'labels': label}, label


def make_input_fn(is_training=True, one_batch=False):
    def input_fn(params):
        batch_size = params['batch_size']
        dataset = tf.data.TFRecordDataset(data_train_file, buffer_size=8*1024*1024)
        dataset = dataset.map(parser).cache().shuffle(batch_size)
        if is_training:
            dataset = dataset.repeat()
        if one_batch:
            dataset = dataset.take(params['batch_size'])
            
        dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(8)

        return dataset
    return input_fn


def predict_input_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(data_test_file).map(parser).cache().shuffle(batch_size)
    dataset = dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True)
    
    return dataset

In [0]:
############################### DATA SAVEING ###################################
 
def images_to_zero_one(images):
        return np.clip(np.array(images) * 0.5 + 0.5, 0., 1.)


def save_imgs(epoch, images):
    if not os.path.exists(GOOGLE_DRIVE_DIR):
        os.mkdir(GOOGLE_DRIVE_DIR)

    # Rescale images to 0 - 1
    images = images_to_zero_one(images)
    fig, axs = plt.subplots(R, C, figsize=(20,20))

    for i in range(R):
        for j in range(C):
            axs[i,j].imshow(images[C*i + j])
            axs[i,j].axis('off')

    fig.savefig(os.path.join(GOOGLE_DRIVE_DIR, '{}.png'.format(epoch)))
    plt.close()

In [0]:
################################## MODEL #######################################

def _relu(x):
    return tf.nn.relu(x)


def _leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=ALPHA)


def _instance_norm(x):
    return tf.contrib.layers.instance_norm(x)


def _dense(x, neurons, name, activation=None):
    return tf.layers.dense(x, neurons, name=name, activation=activation,
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def _conv2d(x, filters, kernel_size, stride, name, activation=None):
    return tf.layers.conv2d(x, filters, [kernel_size, kernel_size], 
                            strides=[stride, stride], activation=activation,
                            padding='same', name=name,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

def _pooling(x, kernel, stride, index):
    return tf.layers.average_pooling2d(x, pool_size=kernel, strides=stride, name='pool_{}'.format(index))

def _deconv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d_transpose(x, filters, [KERNEL_SIZE, KERNEL_SIZE],
                                      strides=[stride, stride], activation=activation,
                                      padding='same', name=name,
                                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def convolution_block(x, filters, kernel_size, resize_factor, index, activation=_relu, activation_first=False, normalization=None):
    if activation and activation_first:
        x = activation(x)
    x = _conv2d(x, kernel_size=kernel_size, filters=filters, stride=resize_factor, activation=None, name='conv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation and not activation_first:
        x = activation(x)
    return x


def deconvolution_block(x, filters, resize_factor, index, activation=_relu, normalization=None):
    x = _deconv2d(x, filters=filters, stride=resize_factor, activation=None, name='deconv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def fully_connected_block(x, neurons, index, activation=_relu, normalization=None):
    x = _dense(x, neurons=neurons, activation=None, name='fc_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def residual_block(x, filters_in, filters_out, index, kernel_size=3, activation=_relu, activation_first=False, normalization=None):
    forwarded_x = convolution_block(x, filters_in, kernel_size, resize_factor=1, 
                                    index='res_con1_{}'.format(index), normalization=normalization,
                                    activation_first=activation_first, activation=activation)
    
    forwarded_x = convolution_block(forwarded_x, filters_out, kernel_size, resize_factor=1, 
                                    index='res_con2_{}'.format(index), normalization=normalization,
                                    activation_first=activation_first, activation=None)
    
    if filters_in != filters_out:
        x = convolution_block(x, filters_out, kernel_size=1, resize_factor=1, index='res_con3_{}'.format(index), activation=None)
    x = tf.add(forwarded_x, x)
    return x


class Funit:

    @staticmethod
    def discriminator(x, label, scope='Discriminator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='disc_10')
            
            x = convolution_block(x, 64, kernel_size=KERNEL_SIZE, resize_factor=2, activation=_leaky_relu, index='disc_11')
            x = convolution_block(x, 128, kernel_size=KERNEL_SIZE, resize_factor=2, activation=_leaky_relu, index='disc_12')
            x = convolution_block(x, 256, kernel_size=KERNEL_SIZE, resize_factor=2, activation=_leaky_relu, index='disc_13')
            x = convolution_block(x, 512, kernel_size=KERNEL_SIZE, resize_factor=2, activation=_leaky_relu, index='disc_14')
                        
            output = tf.layers.flatten(x)
            output = _dense(output, NUM_CLASSES, 'disc_out')
            
#             output = tf.gather_nd(output, tf.expand_dims(label, axis=1), batch_dims=1)
            
            return output, x

In [0]:
################################ MODEL FUN #####################################
def make_model_fn(model):

    def model_fn(features, labels, mode, params):

        with tf.variable_scope('inputs'):
            style_images = features['style_images']
            labels = features['labels']
        d_on_data_labels = tf.one_hot(labels, depth=NUM_CLASSES, axis=-1)
        logits, _ = model.discriminator(style_images, labels)
        
        # PREDICT #
        if mode == tf.estimator.ModeKeys.PREDICT:
          
            predictions = {
                'd_style': tf.argmax(tf.sigmoid(logits), axis=-1),
                'd_style_one_hot': tf.sigmoid(logits),
                'one_hot_labels': d_on_data_labels,
                'labels': labels
            }
            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

          
        # Discriminator loss
        
        print()
        d_on_data_logits = tf.squeeze(logits)
        d_loss = tf.losses.sigmoid_cross_entropy(
            d_on_data_labels,
            d_on_data_logits,
            reduction=tf.losses.Reduction.NONE,
        )

        # TRAIN #
        if mode == tf.estimator.ModeKeys.TRAIN:
            d_loss = tf.reduce_mean(d_loss)
            d_optimizer = tf.train.AdamOptimizer(learning_rate=D_LR, beta1=0.5)
            d_optimizer = tf.contrib.tpu.CrossShardOptimizer(d_optimizer)
         
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                d_step = d_optimizer.minimize(d_loss, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                 scope='Discriminator'))
                increment_step = tf.assign_add(tf.train.get_or_create_global_step(), 1)
                joint_op = tf.group([d_step, increment_step])

                a = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss, train_op=joint_op)
                return a

        # EVAL #
        elif mode == tf.estimator.ModeKeys.EVAL:
            def _eval_metric_fn(d_loss, d_real_labels, d_real_logits):
                return {
                    'discriminator_loss': tf.metrics.mean(d_loss),
                    'discriminator_real_accuracy': tf.metrics.accuracy(labels=d_real_labels, predictions=tf.math.argmax(tf.sigmoid(d_real_logits), axis=-1)),
                }

            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=tf.reduce_mean(d_loss),
                                                   eval_metrics=(_eval_metric_fn, [d_loss, labels, d_on_data_logits]))
    return model_fn

In [0]:
################################ ESTIMATORS ####################################

def make_estimators(model, only_cpu=False):
    model_fn = make_model_fn(model)
    
    config = tf.contrib.tpu.RunConfig(
        master=TF_MASTER,
        save_checkpoints_steps=EVAL_EPOCHS,
        save_checkpoints_secs=None,
        save_summary_steps=EVAL_EPOCHS,
        model_dir=MODEL_DIR,
        keep_checkpoint_max=3,
        tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=EVAL_EPOCHS))

    if not only_cpu:
        # TPU-based estimator used for TRAIN and EVAL
        est = tf.contrib.tpu.TPUEstimator(
            model_fn=model_fn,
            use_tpu=True,
            config=config,
            train_batch_size=BATCH_SIZE,
            eval_batch_size=BATCH_SIZE)
    else:
        est = None

    # CPU-based estimator used for PREDICT (generating images)
    cpu_est = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=False,
        config=config,
        predict_batch_size=EXAMPLES)
    
    return est, cpu_est

In [0]:
################################# TRAINING #####################################

def train(est, cpu_est):
    current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
    print('Starting training')

    while current_step < EPOCHS:
        next_checkpoint = int(min(current_step + EVAL_EPOCHS, EPOCHS))
        est.train(input_fn=make_input_fn(), max_steps=next_checkpoint)
        current_step = next_checkpoint
        print('Finished training step %d' % current_step)

        # Evaluation
        metrics = est.evaluate(input_fn=make_input_fn(False), steps=1)
        print('Finished evaluating')
        print(metrics)
        
        generated_iter = cpu_est.predict(input_fn=make_input_fn(False, True))
        for p in generated_iter:
            print (p['labels'], p['one_hot_labels'], p['d_style'], p['d_style_one_hot'])
        

#         Render some generated images
#         generated_iter = cpu_est.predict(input_fn=predict_input_fn)
#         images = [np.concatenate([p['content_images'], p['style_images'], p['generated_images']], axis=1) for p in generated_iter]
#         save_imgs(str(current_step), images)
#         print('Finished generating images')

In [0]:
def do_experiment():
    setup()
    upload_credentials()
    model = Funit()
    est, cpu_est = make_estimators(model)
    train(est, cpu_est)

In [11]:
tf.logging.set_verbosity(tf.logging.INFO)

try:
    do_experiment()
except Exception as e:
    print (e)
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


W0905 12:59:48.281093 139779565938560 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0905 12:59:49.330157 139779565938560 estimator.py:1984] Estimator's model_fn (<function make_model_fn.<locals>.model_fn at 0x7f20d4bd2ea0>) includes params argument, but params are not passed to Estimator.
I0905 12:59:49.332799 139779565938560 estimator.py:209] Using config: {'_model_dir': 'gs://tputestingmnist/FUNIT/FUNIT_only_disc_2/', '_tf_random_seed': None, '_save_summary_steps': 1000, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
 

Starting training


W0905 12:59:50.763833 139779565938560 deprecation.py:323] From <ipython-input-6-78cd512322f7>:23: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0905 12:59:51.178711 139779565938560 deprecation.py:323] From <ipython-input-6-78cd512322f7>:91: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0905 12:59:51.367083 139779565938560 deprecation.py:323] From <ipython-input-6-78cd512322f7>:16: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0905 12:59:51.626550 139779565938560 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops

I0905 12:59:53.015143 139779565938560 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0905 12:59:53.046429 139779565938560 estimator.py:1147] Done calling model_fn.
I0905 12:59:54.269732 139779565938560 tpu_estimator.py:499] TPU job name tpu_worker
I0905 12:59:54.410843 139779565938560 monitored_session.py:240] Graph was finalized.
I0905 12:59:54.763472 139779565938560 session_manager.py:500] Running local_init_op.
I0905 12:59:54.796020 139779565938560 session_manager.py:502] Done running local_init_op.
I0905 12:59:55.858676 139779565938560 basic_session_run_hooks.py:606] Saving checkpoints for 0 into gs://tputestingmnist/FUNIT/FUNIT_only_disc_2/model.ckpt.
W0905 12:59:59.521096 139779565938560 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py:741: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assi

...

Finished training step 20000



I0905 13:46:42.845729 139779565938560 estimator.py:1147] Done calling model_fn.
I0905 13:46:42.867555 139779565938560 evaluation.py:255] Starting evaluation at 2019-09-05T13:46:42Z
I0905 13:46:42.868808 139779565938560 tpu_estimator.py:499] TPU job name tpu_worker
I0905 13:46:42.967900 139779565938560 monitored_session.py:240] Graph was finalized.
I0905 13:46:43.012037 139779565938560 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_disc_2/model.ckpt-20000
I0905 13:46:44.025523 139779565938560 session_manager.py:500] Running local_init_op.
I0905 13:46:44.046425 139779565938560 session_manager.py:502] Done running local_init_op.
I0905 13:46:44.092725 139779565938560 tpu_estimator.py:557] Init TPU system
I0905 13:47:50.036661 139779565938560 tpu_estimator.py:566] Initialized TPU in 65 seconds
I0905 13:47:50.038476 139778288514816 tpu_estimator.py:514] Starting infeed thread controller.
I0905 13:47:50.040008 139778280122112 tpu_estimator.py:533] Starting outf

Finished evaluating
{'discriminator_loss': 0.00028299377, 'discriminator_real_accuracy': 1.0, 'loss': 0.00030486545, 'global_step': 20000}


I0905 13:47:52.558072 139779565938560 estimator.py:1147] Done calling model_fn.
I0905 13:47:52.643446 139779565938560 monitored_session.py:240] Graph was finalized.
I0905 13:47:52.690633 139779565938560 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_disc_2/model.ckpt-20000
I0905 13:47:52.875625 139779565938560 session_manager.py:500] Running local_init_op.
I0905 13:47:52.883342 139779565938560 session_manager.py:502] Done running local_init_op.
I0905 13:47:52.996634 139779565938560 error_handling.py:96] prediction_loop marked as finished
I0905 13:47:52.997723 139779565938560 error_handling.py:96] prediction_loop marked as finished


5 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 5 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.5372715e-05
 9.9955583e-01 3.9160252e-05 5.9902668e-06 0.0000000e+00 0.0000000e+00
 2.0861626e-07 3.2782555e-07 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 3.8743019e-07
 7.3909760e-06]
2 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 2 [0.0000000e+00 0.0000000e+00 9.5970857e-01 5.9604645e-08 1.5825033e-05
 0.0000000e+00 0.0000000e+00 5.9604645e-08 0.0000000e+00 0.0000000e+00
 0.0000000e+00 6.5565109e-06 0.0000000e+00 0.0000000e+00 0.0000000e+00
 2.0563602e-06 5.5047870e-04 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
7 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 7 [1.7583370e-06 0.0000000e+00 4.5922399e-04 0.0000000e+00 0.0000000e+00
 1.7881393e-07 9.9718571e-05 9.9927330e-01 0.0000000e+00 0.0000000e+00
 2.9802322e-08 8.9406967e-08 0.0000000e+00 2.980232